In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical 

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.optimizers import Adam

from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.callbacks import EarlyStopping
import tensorflow as tf

In [2]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

In [3]:
# X = train.iloc[:,1:]
# Y = train.iloc[:,0]

#splitting dataframe using train_test_split
# x_train , x_test , y_train , y_test = train_test_split(X_train, Y , test_size=0.1, random_state=42)

In [4]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)
# df_test=df_test.values.reshape(-1,28,28,1)

In [5]:
#notice num_classes is set to 10 as we have 10 different labels
y_train = to_categorical(y_train, num_classes=10)
y_test = to_categorical(y_test, num_classes=10)


In [6]:
X_train.shape

(60000, 28, 28, 1)

In [7]:
model = Sequential()

model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last',
                 input_shape=(28,28,1)))
model.add(BatchNormalization())
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=2, padding='valid' ))
model.add(Dropout(0.25))

model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=1, padding='same', data_format='channels_last'))
model.add(BatchNormalization())
model.add(Conv2D(filters=64, kernel_size=(3, 3), strides=1, padding='same', activation='relu', data_format='channels_last'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), padding='valid', strides=2))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(1024, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

In [8]:
optimizer = Adam(lr=0.001, beta_1=0.9, beta_2=0.999 )
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])


In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 batch_normalization (BatchN  (None, 28, 28, 32)       128       
 ormalization)                                                   
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        9248      
                                                                 
 batch_normalization_1 (Batc  (None, 28, 28, 32)       128       
 hNormalization)                                                 
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                        

In [10]:
#for our case LearningRateScheduler will work great
reduce_lr = LearningRateScheduler(lambda x: 1e-3 * 0.9 ** x)

In [11]:
#visualizing what the learning rate decay will do to the learning rate through every epoch
decays = [(lambda x: 1e-3 * 0.9 ** x)(x) for x in range(10)]
i=1

for lr in decays:
    print("Epoch " + str(i) +" Learning Rate: " + str(lr))
    i+=1

Epoch 1 Learning Rate: 0.001
Epoch 2 Learning Rate: 0.0009000000000000001
Epoch 3 Learning Rate: 0.0008100000000000001
Epoch 4 Learning Rate: 0.0007290000000000002
Epoch 5 Learning Rate: 0.0006561000000000001
Epoch 6 Learning Rate: 0.00059049
Epoch 7 Learning Rate: 0.000531441
Epoch 8 Learning Rate: 0.0004782969000000001
Epoch 9 Learning Rate: 0.0004304672100000001
Epoch 10 Learning Rate: 0.0003874204890000001


In [12]:
#by default this is evaluated on 'val_loss'
early_stopping = EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=20, # how many epochs to wait before stopping
    restore_best_weights=True,
)

In [13]:
datagen = ImageDataGenerator(
            featurewise_center=False,  # set input mean to 0 over the dataset
            samplewise_center=False,  # set each sample mean to 0
            featurewise_std_normalization=False,  # divide inputs by std of the dataset
            samplewise_std_normalization=False,  # divide each input by its std
            zca_whitening=False,  # apply ZCA whitening
            rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
            zoom_range = 0.1, # Randomly zoom image 
            width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=False,  # randomly flip images
            vertical_flip=False)  # randomly flip images

In [14]:
batch_size = 64
epochs = 24

# Fit the Model
history = model.fit_generator(datagen.flow(X_train, y_train, batch_size = batch_size), epochs = epochs, 
                              validation_data = (X_test, y_test), verbose=1, 
                              steps_per_epoch=X_train.shape[0] // batch_size,
                              callbacks = [reduce_lr]) #left out early_stopping parameter as it gets better accuracy

C:\ProgramData\anaconda3\envs\tensorF\lib\site-packages\ipykernel_launcher.py:8: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Epoch 1/24
937/937 [==============================] - 699s 734ms/step - loss: 0.3009 - accuracy: 0.9116 - val_loss: 0.0468 - val_accuracy: 0.9859 - lr: 0.0010
Epoch 2/24
937/937 [==============================] - 805s 859ms/step - loss: 0.1033 - accuracy: 0.9687 - val_loss: 0.0429 - val_accuracy: 0.9843 - lr: 9.0000e-04
Epoch 3/24
937/937 [==============================] - 757s 808ms/step - loss: 0.0836 - accuracy: 0.9748 - val_loss: 0.0300 - val_accuracy: 0.9910 - lr: 8.1000e-04
Epoch 4/24
937/937 [==============================] - 738s 788ms/step - loss: 0.0698 - accuracy: 0.9793 - val_loss: 0.0245 - val_accuracy: 0.9919 - lr: 7.2900e-04
Epoch 5/24
937/937 [==============================] - 723s 772ms/step - loss: 0.0609 - accuracy: 0.9822 - val_loss: 0.0252 - val_accuracy: 0.9914 - lr: 6.5610e-04
Epoch 6/24
937/937 [==============================] - 813s 867ms/step - loss: 0.0533 - accuracy: 0.9831 - val_loss: 0.0324 - val_accuracy: 0.9895 - lr: 5.9049e-04
Epoch 7/24
937/937 [======

In [15]:
pred_digits_test = np.argmax(model.predict(X_test),axis=1)
image_id_test = []

for i in range (len(pred_digits_test)):
    image_id_test.append(i+1)
    
d = {'ImageId':image_id_test,'Label':pred_digits_test}
answer = pd.DataFrame(d)

answer.to_csv('answer.csv',index=False)

313/313 [==============================] - 27s 82ms/step


In [16]:
pred_digits_test = to_categorical(pred_digits_test, num_classes=10)

In [17]:
from sklearn.metrics import r2_score, classification_report

In [18]:
r2_score(pred_digits_test, y_test)

0.9913533763263501

In [19]:
print(classification_report(pred_digits_test, y_test))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       981
           1       1.00      1.00      1.00      1135
           2       1.00      1.00      1.00      1030
           3       1.00      0.99      1.00      1016
           4       1.00      1.00      1.00       982
           5       0.99      1.00      1.00       889
           6       0.99      1.00      1.00       955
           7       1.00      0.99      0.99      1030
           8       1.00      1.00      1.00       975
           9       0.99      1.00      0.99      1007

   micro avg       1.00      1.00      1.00     10000
   macro avg       1.00      1.00      1.00     10000
weighted avg       1.00      1.00      1.00     10000
 samples avg       1.00      1.00      1.00     10000



In [20]:
model.save('mnist_class_cnn.h5')